**Extract User-Submitted CellRanger Sample-Level Data from OSD/GLDS-352.**


**CellRanger command:**
- From OSD-352 user-submitted data
- Used Cellranger Arc 2.0.0 for data processing
- The code for the complete analysis pipeline is provided under the github repository: https://github.com/giacomellolab/NASA_RR3_Brain/tree/main/multiomics

### Import libraries

In [ ]:
import h5py
import scipy.io
import numpy as np
import pandas as pd
import os

### Extract GEX barcodes from all cells from each sample

In [ ]:
# List of sample file names and corresponding output file names
sample_files = [
    'CF1_per_barcode_metrics.csv',
    'CF2_per_barcode_metrics.csv',
    'CF7_per_barcode_metrics.csv',
    'CG8_per_barcode_metrics.csv',
    'CG9_per_barcode_metrics.csv'
]

output_files = [
    'CF1_gex_barcodes_for_cells.csv',
    'CF2_gex_barcodes_for_cells.csv',
    'CF7_gex_barcodes_for_cells.csv',
    'CG8_gex_barcodes_for_cells.csv',
    'CG9_gex_barcodes_for_cells.csv'
]

# Process each sample file
for sample_file, output_file in zip(sample_files, output_files):
    # Load the metadata CSV file
    metadata_df = pd.read_csv(sample_file)
    
    # Filter rows where 'is_cell' is 1
    cell_metadata_df = metadata_df[metadata_df['is_cell'] == 1]
    
    # Extract the 'gex_barcode' column
    gex_barcodes = cell_metadata_df['gex_barcode']
    
    # Verify the number of cells
    print(f"Number of cells in {sample_file}: {len(gex_barcodes)}")  # Should match the expected count
    
    # Save the gex_barcodes to a file
    gex_barcodes.to_csv(output_file, index=False, header=False)
    print(f"Saved {output_file}")

# After running the code, the gex barcode files will be saved in the current directory.

### Set data paths

In [ ]:
# Paths to your files
h5_filename = 'GLDS-352_snRNA-Seq_filtered_feature_bc_matrix.h5'  # Replace with your .h5 file path
csv_filenames = [
    'CF1_gex_barcodes_for_cells.csv',
    'CF2_gex_barcodes_for_cells.csv',
    'CF7_gex_barcodes_for_cells.csv',
    'CG8_gex_barcodes_for_cells.csv',
    'CG9_gex_barcodes_for_cells.csv'
]
sample_names = ['CF1', 'CF2', 'CF7', 'CG8', 'CG9']
output_base_dir = 'CR_output'  # Replace with your desired output base directory

### Set up functions to extract sample-level data

In [ ]:
# Function to read HDF5 file
def read_h5_file(h5_filename):
    with h5py.File(h5_filename, 'r') as f:
        barcodes = f['matrix']['barcodes'][()]
        barcodes = [barcode.decode('utf-8') for barcode in barcodes]
        
        features = f['matrix']['features']['name'][()]
        feature_ids = f['matrix']['features']['id'][()]
        feature_types = f['matrix']['features']['feature_type'][()]
        
        data = f['matrix']['data'][()]
        indices = f['matrix']['indices'][()]
        indptr = f['matrix']['indptr'][()]
        shape = f['matrix']['shape'][()]
        
    return barcodes, features, feature_ids, feature_types, data, indices, indptr, shape

# Function to write TSV and MTX files
def write_files(barcodes, features, feature_ids, feature_types, data, indices, indptr, shape, sample_barcodes, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    
    sample_indices = [i for i, barcode in enumerate(barcodes) if barcode in sample_barcodes]
    filtered_data = []
    filtered_indices = []
    filtered_indptr = [0]

    for idx in sample_indices:
        start = indptr[idx]
        end = indptr[idx + 1]
        filtered_data.extend(data[start:end])
        filtered_indices.extend(indices[start:end])
        filtered_indptr.append(len(filtered_data))
    
    filtered_shape = (shape[0], len(sample_indices))

    with open(os.path.join(output_dir, 'barcodes.tsv'), 'w') as bfile:
        bfile.write('\n'.join(sample_barcodes) + '\n')

    with open(os.path.join(output_dir, 'features.tsv'), 'w') as ffile:
        for i in range(len(features)):
            ffile.write(f"{feature_ids[i].decode('utf-8')}\t{features[i].decode('utf-8')}\t{feature_types[i].decode('utf-8')}\n")

    scipy.io.mmwrite(os.path.join(output_dir, 'matrix.mtx'), 
                     scipy.sparse.csc_matrix((filtered_data, filtered_indices, filtered_indptr), shape=filtered_shape))

# Function to process samples
def process_samples(h5_filename, csv_filenames, sample_names, output_base_dir):
    barcodes, features, feature_ids, feature_types, data, indices, indptr, shape = read_h5_file(h5_filename)
    
    for csv_filename, sample_name in zip(csv_filenames, sample_names):
        df = pd.read_csv(csv_filename, header=None)
        sample_barcodes = df[0].tolist()
        
        output_dir = os.path.join(output_base_dir, sample_name)
        write_files(barcodes, features, feature_ids, feature_types, data, indices, indptr, shape, sample_barcodes, output_dir)

### Extract sample-level data

In [ ]:
# Process samples
process_samples(h5_filename, csv_filenames, sample_names, output_base_dir)

# Display output files for verification
for sample_name in sample_names:
    print(f"Files for sample {sample_name}:")
    sample_output_dir = os.path.join(output_base_dir, sample_name)
    for root, dirs, files in os.walk(sample_output_dir):
        for file in files:
            print(os.path.join(root, file))
    print()

### Check number of cells in the original filtered_feature_bc_matrix.h5 file compared with the sum of the number of cells extracted for each sample

In [ ]:
# Function to read barcodes from HDF5 file
def count_barcodes_in_h5(h5_filename):
    with h5py.File(h5_filename, 'r') as f:
        barcodes = f['matrix']['barcodes'][()]
        return len(barcodes)

# Function to count lines in a file (used for barcodes.tsv)
def count_lines(file_path):
    with open(file_path, 'r') as f:
        return sum(1 for _ in f)

# Path to your original HDF5 file
h5_filename = 'GLDS-352_snRNA-Seq_filtered_feature_bc_matrix.h5'  # Replace with your .h5 file path

# Paths to your extracted barcodes.tsv files for each sample
sample_dirs = [
    'CR_output/CF1',
    'CR_output/CF2',
    'CR_output/CF7',
    'CR_output/CG8',
    'CR_output/CG9'
]

# Count cells in the original HDF5 file
original_cell_count = count_barcodes_in_h5(h5_filename)
print(f"Number of cells in the original HDF5 file: {original_cell_count}")

# Count cells in the extracted barcodes.tsv files for each sample
extracted_cell_count = 0
for sample_dir in sample_dirs:
    barcodes_file = os.path.join(sample_dir, 'barcodes.tsv')
    sample_cell_count = count_lines(barcodes_file)
    extracted_cell_count += sample_cell_count
    print(f"Number of cells in {barcodes_file}: {sample_cell_count}")

# Print total number of cells in extracted files
print(f"Total number of cells in extracted files: {extracted_cell_count}")

# Compare counts
if original_cell_count == extracted_cell_count:
    print("The number of cells matches between the original HDF5 file and the extracted files.")
else:
    print("The number of cells does not match. Please check the extracted files and the original HDF5 file.")


*Note: If these number do not match exactly it could be due to different filtering parameters for GEX and ATACseq barcodes.*

### Check that the numbers in the first row of the matrix.mtx file matches the number of features (aka genes) in the features.tsv, the number of barcodes (aka cells) in the barcodes.tsv, and the number of UMIs in the matrix.mtx file, respectively, for each sample

In [ ]:
def check_files(sample_dir):
    barcodes_file = os.path.join(sample_dir, 'barcodes.tsv')
    features_file = os.path.join(sample_dir, 'features.tsv')
    matrix_file = os.path.join(sample_dir, 'matrix.mtx')

    # Step 1: Read the barcodes.tsv file and count the rows
    with open(barcodes_file, 'r') as f:
        barcodes_count = sum(1 for line in f)
    
    # Step 2: Read the features.tsv file and count the rows
    with open(features_file, 'r') as f:
        features_count = sum(1 for line in f)
    
    # Step 3: Read the matrix.mtx file and get the values in the first row
    with open(matrix_file, 'r') as f:
        for line in f:
            if not line.startswith('%'):
                first_line = line.strip()
                break
        matrix_info = first_line.split()
        if len(matrix_info) >= 3:
            matrix_features_count = int(matrix_info[0])
            matrix_barcodes_count = int(matrix_info[1])
            matrix_data_count = int(matrix_info[2])
        else:
            raise ValueError("The first row of the matrix.mtx file does not contain the expected number of columns.")

    # Step 4: Read the matrix.mtx file and count the rows of data
    with open(matrix_file, 'r') as f:
        data_count = sum(1 for line in f if not line.startswith('%')) - 1  # subtract 1 to exclude the header row

    # Step 5: Compare the counts
    if barcodes_count == matrix_barcodes_count and features_count == matrix_features_count and data_count == matrix_data_count:
        print(f"Success: The number of rows in barcodes.tsv ({barcodes_count}), features.tsv ({features_count}), and matrix.mtx data rows ({data_count}) match the values in the matrix.mtx header for {sample_dir}.")
    else:
        print(f"Error: There is a mismatch in {sample_dir}.")
        if barcodes_count != matrix_barcodes_count:
            print(f"  - Barcodes count: {barcodes_count}, expected: {matrix_barcodes_count}")
        if features_count != matrix_features_count:
            print(f"  - Features count: {features_count}, expected: {matrix_features_count}")
        if data_count != matrix_data_count:
            print(f"  - Matrix data rows count: {data_count}, expected: {matrix_data_count}")

def check_all_samples(base_dir, samples):
    for sample in samples:
        sample_dir = os.path.join(base_dir, sample)
        check_files(sample_dir)

# Usage
base_dir = 'CR_output'
samples = ['CF1', 'CF2', 'CF7', 'CG8', 'CG9']
check_all_samples(base_dir, samples)